In [0]:
import logging
from settings import create_spark_session
from pyspark.sql.functions import lit

In [0]:
def read_csv_with_validation(spark, path, ingest_date):
    try:
        df = (
            spark.read
            .option("header", True)
            .option("inferSchema", True)
            .option("badRecordsPath", f"/tmp/bad_records/{ingest_date}")
            .csv(path)
        )
        df = df.withColumn("ingest_date", lit(ingest_date))
        logger.info(f"Leitura concluída para {path} com {df.count()} registros.")
        return df
    except Exception as e:
        logger.error(f"Erro ao ler {path}: {e}")
        return None

def save_table(df, table_name, bronze_schema, partition_col):
    df.write.format("delta") \
        .mode("append") \
        .partitionBy(partition_col) \
        .saveAsTable(f"{bronze_schema}.{table_name}")

def main(spark, raw_path, bronze_schema, ingest_date):

    logger.info("Iniciando ingestão na camada Bronze...")

    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {bronze_schema}") # criação do schema se não existir

    tables = ["orders", "customers", "products", "order_items", "inventory_updates"]

    for table in tables:
        file_path = f"{raw_path}/{table}.csv"
        logger.info(f"Lendo a tabela {table} de {file_path}")
        df = read_csv_with_validation(spark, file_path, ingest_date)

        if df:
            table_name = table
            try:
                save_table(df, table_name, bronze_schema, "ingest_date")
                logger.info(f"Dados gravados com sucesso em {table_name}")

            except Exception as e:
                logger.error(f"Erro ao gravar Delta para {table}: {e}")
        else:
            logger.warning(f"Tabela {table} não foi processada.")

    logger.info("Finalizando sessão spark.")
    spark.stop()
    
    logger.info("Ingestão Bronze finalizada com sucesso.")

In [0]:

# Configuração do Logger
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger("bronze_layer")

bronze_config = {
    "spark.sql.shuffle.partitions": "100",
    "spark.databricks.delta.optimizeWrite.enabled": "true",
    "spark.databricks.delta.autoCompact.enabled": "true",
    "spark.sql.parquet.filterPushdown": "true",
    "spark.sql.parquet.mergeSchema": "false",
    "spark.sql.files.ignoreCorruptFiles": "true",
    "spark.sql.files.ignoreMissingFiles": "true",
}

app_name = "BronzeLayer"
spark = create_spark_session(app_name, bronze_config)

# Variaveis
ingest_date = "2025-06-15"
raw_path = f"/Volumes/workspace/lakehouse/a_raw/{ingest_date}"
bronze_schema = "lakehouse.a_bronze"

main(spark, raw_path, bronze_schema, ingest_date)

2025-06-15 13:45:12,648 - INFO - Iniciando ingestão na camada Bronze...
2025-06-15 13:45:12,650 - INFO - Lendo a tabela orders de /Volumes/workspace/lakehouse/raw/2025-06-15/orders.csv
2025-06-15 13:45:13,870 - INFO - Leitura concluída para /Volumes/workspace/lakehouse/raw/2025-06-15/orders.csv com 300000 registros.
2025-06-15 13:45:17,687 - INFO - Dados gravados com sucesso em orders
2025-06-15 13:45:17,688 - INFO - Lendo a tabela customers de /Volumes/workspace/lakehouse/raw/2025-06-15/customers.csv
2025-06-15 13:45:18,808 - INFO - Leitura concluída para /Volumes/workspace/lakehouse/raw/2025-06-15/customers.csv com 196130 registros.
2025-06-15 13:45:22,340 - INFO - Dados gravados com sucesso em customers
2025-06-15 13:45:22,341 - INFO - Lendo a tabela products de /Volumes/workspace/lakehouse/raw/2025-06-15/products.csv
2025-06-15 13:45:23,191 - INFO - Leitura concluída para /Volumes/workspace/lakehouse/raw/2025-06-15/products.csv com 4904 registros.
2025-06-15 13:45:25,785 - INFO - D

Explicação das configs para a camada bronze
  - **spark.sql.shuffle.partitions = 100**
    - Ingestões da Bronze geralmente envolvem menos transformações, mas ainda podem haver repartições.
    - Um número menor (comparado à Silver e Gold) ajuda a evitar overhead desnecessário.

  - **spark.databricks.delta.optimizeWrite.enabled = true**
    - Mantém a escrita mais eficiente mesmo com dados brutos.
    - Reduz a criação de muitos arquivos pequenos.

  - **spark.databricks.delta.autoCompact.enabled = true**
    - Ajuda a consolidar arquivos pequenos automaticamente.
    - Importante para evitar "file explosion" após ingestões frequentes ou em micro-batches.

  - **spark.sql.parquet.filterPushdown = true**
    - Já ativa desde a Bronze para aproveitar filtros mesmo nos dados brutos quando possível.
    - Economiza I/O mesmo antes das transformações.

  - **spark.sql.parquet.mergeSchema = false**
    - Evita sobrecarga desnecessária tentando unificar esquemas automaticamente.
    - Na Bronze, o schema deve ser fixo ou controlado, não tolerando variações inesperadas.

  - **spark.sql.files.ignoreCorruptFiles = true**
    - Permite que arquivos corrompidos não interrompam o processo de ingestão.
    - Crítico para garantir robustez em ambientes de ingestão contínua.

  - **spark.sql.files.ignoreMissingFiles = true**
    - Similar ao anterior: evita falhas se algum arquivo referenciado tiver sido excluído ou estiver faltando.

Essa configuração prioriza:
  - Tolerância a falhas → evita que arquivos ruins interrompam o pipeline
  - Escrita eficiente → com Delta otimizado e compactação
  - Leitura segura → com pushdown e controle rígido de schema